In [7]:
import pandas as pd
from azure.core.credentials import AzureKeyCredential
from  getAnalisis import analizar_texto_azure
from traduccionImg import traducir_texto_ocr

df = pd.read_csv("./export_electronics.csv")


In [3]:
import os
from dotenv import load_dotenv
load_dotenv()
#Inicaliazar traductor de texto y analizador de texto
ENPOINT_URL_TRANSLATOR =  os.getenv("ENPOINT_URL_TRANSLATOR")
CREDENTIAL_TRANSLATOR =  os.getenv("CREDENTIAL_TRANSLATOR")

credential_trans = AzureKeyCredential(
    CREDENTIAL_TRANSLATOR
)
#idioma_detectado, texto_traducido, idioma_destino= traducir_texto_ocr(credential, ENPOINT_URL_TRANSLATOR, texto)
ENPOINT_URL_LANGUAGE =  os.getenv("ENPOINT_URL_LANGUAGE")
CREDENTIAL_LANGUAGE =  os.getenv("CREDENTIAL_LANGUAGE")

credential_anali = AzureKeyCredential(
    CREDENTIAL_LANGUAGE
)

#resultado_texto = analizar_texto_azure(credential, ENPOINT_URL_LANGUAGE, texto)



In [ ]:
df_texto = df[['reviews', 'date']]
df_texto.drop_duplicates()
df_texto.dropna(axis=0, how='any')
df_texto = df_texto[:500]

In [12]:
from tqdm import tqdm 
import time
analisis_resultados = []

# tqdm envuelve el iterador y muestra el progreso
for _, row in tqdm(df_texto.iterrows(), total=len(df_texto), desc="Analizando reseñas", ncols=90):
    texto = row['reviews']
    date = row['date']

    try:
        _, texto_traducido, _ = traducir_texto_ocr(credential_trans, ENPOINT_URL_TRANSLATOR, texto)
        analisis = analizar_texto_azure(credential_anali, ENPOINT_URL_LANGUAGE, texto_traducido)
    except Exception as e:
        continue    

    analisis_resultados.append({
        "Texto": texto_traducido,
        "Fecha": date,
        "sentimiento": analisis["sentimiento"],
        "etiqueta": analisis["etiqueta"]
    })

    time.sleep(3.0)


🔍 Analizando reseñas: 100%|████████████████████████████| 500/500 [37:38<00:00,  4.52s/it]


In [13]:
# Convertir la lista de diccionarios a un DataFrame
df_resultados = pd.DataFrame(analisis_resultados)

# Guardar en CSV
df_resultados.to_csv("resultados_analisis.csv", index=False, encoding="utf-8-sig")